In [1]:
import importlib
import math
import os
import random
import time

import numpy as np
import boost_histogram as bh
import pandas as pd
import selanneal
from selanneal import annealing

# can disable numba, but is much slower
# os.environ['DISABLE_NUMBA'] = '1'

np.random.seed(42)

#### generate some toy data points in 5 dimensions

In [2]:
features = ['A', 'B', 'C', 'D', 'int_var']
# features = ['A', 'B']

n_dim = len(features)
n_sig = 10_000
n_bkg = 1_000_000

# generated signal events
sig_gen = np.random.multivariate_normal(np.zeros(n_dim), np.identity(n_dim), size=n_sig)
Ngen = len(sig_gen)

# select random subset for reconstructed events
rand_10p = np.random.rand(n_sig) > 0.9
rand_50p = np.random.rand(n_sig) > 0.5
sig_rec = sig_gen[((sig_gen[:, 0] >= 0) & rand_10p) | ((sig_gen[:, 0] < 0) & rand_50p)]

# uniform background between -5 and 5
bkg_rec = np.random.uniform(- np.ones(n_dim) * 5, np.ones(n_dim) * 5, size=[n_bkg, n_dim])

# make int_var boolean
sig_rec[:, 4] = np.abs(sig_rec[:, 4]) < 1.5
bkg_rec[:, 4] = np.abs(bkg_rec[:, 4]) < 1.5

#### create dataframe from generated data

In [3]:
sig_df = pd.DataFrame(sig_rec, columns=features)
sig_df['isSignal'] = 1
sig_df['weight'] = 1.0

bkg_df = pd.DataFrame(bkg_rec, columns=features)
bkg_df['isSignal'] = 0
bkg_df['weight'] = 2.0  # scale to data lumi

df = pd.concat([sig_df, bkg_df], ignore_index=True)


def print_cross_check(result):
    """applies optimised selection and prints FOM/efficiency/purity"""
    query = '(' + ') and ('.join(result[0]) + ')'
    print(query)
    sig_sel = sig_df.query(query)
    bkg_sel = bkg_df.query(query)
    print(f'significance FOM: {(sig_sel.shape[0]) / (np.sqrt(sig_sel.shape[0] + bkg_sel.shape[0] * 2.0)):.3f}')
    print(f'efficiency: {sig_sel.shape[0] / Ngen:.3f}')
    print(f'purity: {sig_sel.shape[0] / (sig_sel.shape[0] + bkg_sel.shape[0] * 2.0):.3f}')

### run the simulated annealing
#### maximise Nsig / sqrt(Nsig + Nbkg)

In [4]:
# reload packages to compile again (in case something unexpected happens)
# importlib.reload(selanneal)
# importlib.reload(annealing)

# need to specify the integer axes and pass the variable name to metadata
# integer features should always come last in the passed data array!
int_axes = [
    bh.axis.IntCategory([0, 1], metadata='int_var'),
]

result = selanneal.optimise.iterate(df[features].values, df['isSignal'].values, features=features, weight=df['weight'].values,
                                    int_axes=int_axes, new_bins=5, Tmin=1E-7, Tmax=1E-2, steps=5_000, quantile=0,  verbosity=0)
print(f'best energy = {-result[-1]}\n')
print_cross_check(result)

Only 0.0 % of new states were accepted in the first iteration. Consider to increase the maximum temperature.
best energy = 20.71122196064876

(-1.5956 <= A < -0.0003) and (-1.4405 <= B < 1.4222) and (-1.5929 <= C < 1.5361) and (-1.6609 <= D < 1.6222) and (1.0000 <= int_var <= 1.0000)
significance FOM: 20.711
efficiency: 0.131
purity: 0.327


#### maximise purity at fixed efficieny

In [5]:
eff_threshold = 0.1
result = selanneal.optimise.iterate(df[features].values, df['isSignal'].values, features=features, weight=df['weight'].values,
                                    int_axes=int_axes, Nexp=Ngen, eff_threshold=eff_threshold, new_bins=5, Tmin=1E-7, Tmax=1E-2, steps=5_000, quantile=0,  verbosity=0)
print(f'best energy = {-result[-1]}\n')
print_cross_check(result)

best energy = 0.3945818610129564

(-1.4656 <= A < -0.0003) and (-1.0412 <= B < 1.1584) and (-1.5946 <= C < 1.4164) and (-1.3963 <= D < 1.4672) and (1.0000 <= int_var <= 1.0000)
significance FOM: 19.914
efficiency: 0.101
purity: 0.395
